In [ ]:
import pandas as pd
import numpy as np

from imblearn.under_sampling import RandomUnderSampler

In [ ]:
!gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet /home/eanegrin/datasets/

In [ ]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
# base_path = 'C:/Eugenio/Maestria/DMEyF/'
base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_fe_v01.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [122219, 109279, 400391, 401537, 999961]

data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)

In [ ]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [ ]:
data['clase_binaria1'] = 0
data['clase_binaria2'] = 0
data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [ ]:
meses_train = [202006,202007,202008,202009,202010,202011,202012,
               202101,202102,202103,202104,202105,202106]

data = data[data['foto_mes'].isin(meses_train)]

In [ ]:
df_subsampled = []

# Iterar sobre cada grupo de 'foto_mes' para realizar el submuestreo
for mes, group in data.groupby("foto_mes"):
    # Separamos características y target
    X = group.drop(columns="clase_binaria2")
    y = group["clase_binaria2"]

    # Aplicar RandomUnderSampler con una estrategia de 10:1
    rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
    X_res, y_res = rus.fit_resample(X, y)

    # Reconstruimos el DataFrame submuestreado del grupo
    group_resampled = pd.concat([X_res, y_res], axis=1)
    group_resampled["foto_mes"] = mes  # Añadir nuevamente la columna 'foto_mes'

    # Añadimos el DataFrame submuestreado a la lista
    df_subsampled.append(group_resampled)

# Concatenamos todos los grupos submuestreados en un único DataFrame
data = pd.concat(df_subsampled, ignore_index=True)

In [ ]:
# Verificar la proporción de clases en el DataFrame submuestreado
class_proportions = df_subsampled["clase_binaria2"].value_counts(normalize=True)